In [8]:
from __future__ import division
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [14]:
import os

cwd = os.getcwd()

parent = os.path.dirname(cwd)

parent

'/Users/juju/Library/Mobile Documents/com~apple~CloudDocs/ta_project'

In [15]:
review = pd.read_csv(os.path.join(parent+r"/data",r"houston_reviews_updated.csv"),sep=',',header=0,infer_datetime_format=True,parse_dates=['review_date','stay_date'])


In [17]:
review["SHARE "]

459562

In [19]:
review[review["review_date"]<='2016-12']["SHARE ID"].unique()

array([   81154,    83265,    84776,    86944,    88957,    90258,
          91472,    93066,    98101,   103740,   104325,   108055,
         112770,   115621,   117504,   117629,   118499,   118809,
         124403,   124580,   168814,   202311,   544043,   585678,
         607825,   722758,  1003761,  1077417,  1229388,  1507266,
        1871564,  1997513,  2028654,  2049952,  2168379,  2189450,
        2678530,  2805362,  2836671,  2909058,  3509458,  3563579,
        3625356,  3716665,  3795983,  3826299,  3996920,  4090295,
        4134719,  4146552,  4193493,  4205816,  4243625,  4271453,
        4279026,  4286929,  4474428,  4481810,  4620189,  4813609,
        4957075,  4994502,  5096287,  5144106,  5144453,  5161047,
        5512973,  5722665,  5796658,  6067146,  6071012,  6079556,
        6158642,  6177422,  6195031,  6234674,  6257309,  6268838,
        6423895,  6483402,  6600479,  6641035,  6684673,  6735630,
        6832197,  6900894,  6930752,  6978856,  7246418,  7272

In [64]:
review["collect_by"].unique()

array([nan, 'partnership'], dtype=object)

In [65]:
## Run this for Chicago only

review["Location"] = review["Location"]/10
review["Rooms"] = review["Rooms"]/10
review["Service"] = review["Service"]/10
review["Cleanliness"] = review["Cleanliness"]/10
review["Quality"] = review["Quality"]/10
review["Value"] = review["Value"]/10
review["ratings"] = review["ratings"]/10

# Aggregate reviews to monthly ratings

In [66]:
review["post_month"] = review["review_date"].dt.to_period('M')
review["stay_month"] = review["stay_date"].dt.to_period('M')
review["renovation"] = review["renovation"].astype(str)

In [67]:
# percentage of reviews are not posted during the stay month
len(review[review["stay_month"]!=review["post_month"]])/len(review)

0.3027342508254462

In [68]:
# Average delay of posting among delayed reviews 
s = review[review["stay_month"]!=review["post_month"]]["post_month"]-review[review["stay_month"]!=review["post_month"]]["stay_month"]

type(s.iloc[0])


pandas._libs.tslibs.offsets.MonthEnd

### Average delay of postings 

In [69]:
# Convert pandas.tseries.offsets.MonthEnd to int

# Average delay of posting among delayed reviews 
s = review[review["stay_month"]!=review["post_month"]]["post_month"].astype(int)-review[review["stay_month"]!=review["post_month"]]["stay_month"].astype(int)
s.describe()

count    71425.000000
mean         2.457291
std          3.173667
min        -26.000000
25%          1.000000
50%          1.000000
75%          2.000000
max         62.000000
dtype: float64

## Aggregate to Monthly Ratings

In [70]:
review.head(5)

,Location,Rooms,Service,Cleanliness,Quality,Value,collect_by,hotel_response,ratings,review_date,user_review_hist_x,stayed_as_month,stayed_as_type,review_month,SHARE ID,renovation,stay_date,post_month,stay_month
0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,True,5.0,2014-08-28,2.0,NaN,NaN,2014-08,83784,nan,2014-08-28,2014-08,2014-08
1,0.0,0.0,0.0,0.0,0.0,0.0,NaN,False,4.0,2008-04-08,122.0,NaN,NaN,2008-04,84915,nan,2008-04-08,2008-04,2008-04
2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,False,5.0,2008-06-10,62.0,NaN,NaN,2008-06,84915,nan,2008-06-10,2008-06,2008-06
3,4.0,5.0,5.0,5.0,0.0,4.0,NaN,False,5.0,2010-04-17,36.0,NaN,NaN,2010-04,84915,nan,2010-04-17,2010-04,2010-04
4,4.0,5.0,5.0,5.0,0.0,5.0,NaN,False,5.0,2010-08-19,25.0,NaN,NaN,2010-08,84915,nan,2010-08-19,2010-08,2010-08


In [71]:

hotels=review.groupby("SHARE ID")

# Avoid divide by Zero

def avoid_zero_denominator_mean(x):
    try:
        y = sum(x)/sum(x!=0)
    except ZeroDivisionError:
        y = 0
    return y

# Aggregate by review post month
l=[]
for name,hotel in hotels:
    aggregations={
    "ratings":["mean","count","sum",
               lambda x:sum(x==1),
               lambda x: sum(x==2), 
               lambda x: sum(x==3),
               lambda x: sum(x==4),
               lambda x: sum(x==5)
              ],
    "hotel_response":"count",
    "collect_by":lambda x: sum(x == "partnership"),
    "Cleanliness":[lambda x: sum(x!=0),lambda x: avoid_zero_denominator_mean(x)],
    "Location":[lambda x: sum(x!=0),lambda x: avoid_zero_denominator_mean(x)],
    "Quality":[lambda x: sum(x!=0),lambda x: avoid_zero_denominator_mean(x)],
    "Rooms":[lambda x: sum(x!=0),lambda x: avoid_zero_denominator_mean(x)],
    "Service":[lambda x: sum(x!=0),lambda x: avoid_zero_denominator_mean(x)],
    "Value":[lambda x: sum(x!=0),lambda x: avoid_zero_denominator_mean(x)],
    "renovation":lambda x: (sum(x == 'True')!=0),
    "stayed_as_type":[lambda x: sum(x.str.contains("solo")),
                     lambda x: sum(x.str.contains("couple")),
                     lambda x: sum(x.str.contains("family")),
                     lambda x: sum(x.str.contains("business"))]
    }
    d = hotel.groupby("post_month").agg(aggregations)
    d.columns= ["monthly_rating_mean",
               "monthly_review_count",
                "monthly_rating_sum",
               "monthly_one_star_count",
               "monthly_two_star_count",
                "monthly_three_star_count",
                "monthly_four_star_count",
                "monthly_five_star_count",
               "monthly_hotel_response_count",
               "monthly_partnership_reviews_count",
               "cleanliness_count",
                "cleanliness_mean",
                "location_count",
                "location_mean",
                "sleep_quality_count",
                "sleep_quality_mean",
                "room_quality_count",
                "room_quality_mean",
                "service_count",
                "service_mean",
                "value_count",
                "value_mean",
                "renovate",
                "solo_count",
                "couple_count",
                "family_count",
                "business_count"]
    
    d = d.reset_index()
    d["SHARE ID"] = name
    l.append(d)
        

df=pd.concat(l,axis=0)

In [72]:
df.columns

Index(['post_month', 'monthly_rating_mean', 'monthly_review_count',
       'monthly_rating_sum', 'monthly_one_star_count',
       'monthly_two_star_count', 'monthly_three_star_count',
       'monthly_four_star_count', 'monthly_five_star_count',
       'monthly_hotel_response_count', 'monthly_partnership_reviews_count',
       'cleanliness_count', 'cleanliness_mean', 'location_count',
       'location_mean', 'sleep_quality_count', 'sleep_quality_mean',
       'room_quality_count', 'room_quality_mean', 'service_count',
       'service_mean', 'value_count', 'value_mean', 'renovate', 'solo_count',
       'couple_count', 'family_count', 'business_count', 'SHARE ID'],
      dtype='object')

In [73]:
#### Some months don't have any reviews
#### Need to correctly fill in the review ratings and review counts for those rows
#### For review count columns, the monthly count needs to be zero 
number_of_reviews_cols = [ 'monthly_rating_mean', 'monthly_rating_sum', 'monthly_review_count', 'monthly_one_star_count',
       'monthly_two_star_count', 'monthly_three_star_count', 'monthly_four_star_count', 'monthly_five_star_count', 
       'monthly_hotel_response_count', 'monthly_partnership_reviews_count',
       'cleanliness_count', 'location_count',
       'sleep_quality_count', 
       'room_quality_count','service_count', 'value_count', 
       'renovate', 
       'solo_count',
       'couple_count', 'family_count', 'business_count']


l=[]

hotels=df.groupby("SHARE ID")

for name, h in hotels:
    h_resample = h.set_index('post_month').resample('M').asfreq().ffill().reset_index(level=0, drop=False)
    h_resample["month_with_reviews"] = h_resample["post_month"].isin(h["post_month"])
    
    for col in number_of_reviews_cols:
    
        h_resample[col]= np.where(h_resample["month_with_reviews"]==False, 
                      0,h_resample[col])
    l.append(h_resample)
    
df_monthly = pd.concat(l,axis=0)

# Aggregate monthly ratings to cummulative ratings

In [74]:
l=[]
g = df_monthly.groupby("SHARE ID")

for name,group in g:
    group["total_rating_sum"] = group["monthly_rating_sum"].expanding().sum()
    group["num_of_reviews"] = group["monthly_review_count"].expanding().sum()
    group["total_one_star"] = group["monthly_one_star_count"].expanding().sum()
    group["total_two_star"] = group["monthly_two_star_count"].expanding().sum()
    group["total_three_star"] = group["monthly_three_star_count"].expanding().sum()
    group["total_four_star"] = group["monthly_four_star_count"].expanding().sum()
    group["total_five_star"] = group["monthly_five_star_count"].expanding().sum()
    group["accum_rating"] = group["total_rating_sum"]/group["num_of_reviews"]
    l.append(group)
    
df_cum= pd.concat(l,axis=0)


In [75]:
df_cum

,post_month,monthly_rating_mean,monthly_review_count,monthly_rating_sum,monthly_one_star_count,monthly_two_star_count,monthly_three_star_count,monthly_four_star_count,monthly_five_star_count,monthly_hotel_response_count,...,SHARE ID,month_with_reviews,total_rating_sum,num_of_reviews,total_one_star,total_two_star,total_three_star,total_four_star,total_five_star,accum_rating
0,2004-10,5.0,1.0,5.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,True,5.0,1.0,0.0,0.0,0.0,0.0,1.0,5.00
1,2004-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,False,5.0,1.0,0.0,0.0,0.0,0.0,1.0,5.00
2,2004-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,False,5.0,1.0,0.0,0.0,0.0,0.0,1.0,5.00
3,2005-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,False,5.0,1.0,0.0,0.0,0.0,0.0,1.0,5.00
4,2005-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,False,5.0,1.0,0.0,0.0,0.0,0.0,1.0,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,2017-06,4.0,1.0,4.0,0.0,0.0,0.0,1.0,0.0,1.0,...,10127842.0,True,21.0,7.0,3.0,0.0,0.0,2.0,2.0,3.00
78,2017-07,5.0,1.0,5.0,0.0,0.0,0.0,0.0,1.0,1.0,...,10127842.0,True,26.0,8.0,3.0,0.0,0.0,2.0,3.0,3.25
79,2017-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10127842.0,False,26.0,8.0,3.0,0.0,0.0,2.0,3.0,3.25
80,2017-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10127842.0,False,26.0,8.0,3.0,0.0,0.0,2.0,3.0,3.25


In [76]:
df_cum["total_one_star_%"] = df_cum["total_one_star"]/df_cum["num_of_reviews"]
df_cum["total_two_star_%"] = df_cum["total_two_star"]/df_cum["num_of_reviews"]
df_cum["total_three_star_%"] = df_cum["total_three_star"]/df_cum["num_of_reviews"]
df_cum["total_four_star_%"] = df_cum["total_four_star"]/df_cum["num_of_reviews"]
df_cum["total_five_star_%"] = df_cum["total_five_star"]/df_cum["num_of_reviews"]

In [77]:
df_cum = df_cum.drop(["monthly_rating_sum","total_rating_sum"],axis=1)

In [78]:
# Save result

output_dir = parent+r"/data/monthly_review_data"
df_cum.to_excel(os.path.join(output_dir,r"Chicago_monthly_data.xlsx"), index = False, encoding='utf-8')